# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [1]:
import mujoco
import mujoco_viewer
import numpy as np
from numpy.linalg import inv, pinv
import scipy as sp

from utilts import rot_err, getSitePose, getSiteVel, getSiteJacobian
from autograder import autograder
from sim import Sim

franka_sim = Sim(fname='./franka_emika_panda/scene_torque_actuators.xml')
atlas_sim = Sim(fname='./atlas/atlas.xml', rand_rgba=True)


In [2]:


def getMassMatrix(model, data):
    """
        This function takes in a mujoco model and data class and returns
        the full system mass/inertia matrix 
        M(q) : nxn matrix where n = dim(q) <- note that for floating base system, this includes 6 dim base degree DoF
    """
    mass_matrix = np.zeros((model.nv, model.nv))
    mujoco.mj_fullM(model, mass_matrix, data.qM)
    return mass_matrix

def getBiasTerms(model, data):
    """
        This function takes in a mujoco model and data class and returns
        the dynamic bias terms e.g., friction, gravity, coriolis, etc. 
        b(q,qdot) : nx1 array
    """
    return data.qfrc_bias

def getOSMassMatrix(sys_mass_matrix, jac):
    """
        This function takes in the mass matrix and a jacobian matrix and returns the operational space matrix M_x(q)
    """
    return inv(jac@np.linalg.inv(sys_mass_matrix)@jac.T)

def getJointSpaceMassMatrix(sys_mass_matrix, selection_matrix):
    return inv(
                selection_matrix @ inv(sys_mass_matrix) @ selection_matrix.T
            )

def getSelectionMatrix(model):
    Sq = np.concatenate([
        np.zeros((model.nv-6, 6)),
        np.eye(model.nv-6)
    ], axis=1)
    return Sq

def getDynConsSelectionMatrix(model, data):
    sys_mass = getMassMatrix(model, data)
    Sq       = getSelectionMatrix(model)
    M        = getJointSpaceMassMatrix(sys_mass, Sq)
    return inv(sys_mass) @ Sq.T @ M 

def getGeneralizedJacobian(model, data):
    jac_sys = getSiteJacobian(model, data, "l_hand_site") 
    Sbarq = getDynConsSelectionMatrix(model, data)
    J = jac_sys @ Sbarq
    return J 

def freebaseOSMatrix(model, data):
    mass_matrix = getMassMatrix(model, data)
    Abb = mass_matrix[0:6,0:6]
    Abr = mass_matrix[0:6,6:]
    Arb = mass_matrix[6:,0:6]
    Arr = mass_matrix[6:,6:]
    J_full = getSiteJacobian(model, data, "l_hand_site")
    Sq_bar = np.vstack((np.linalg.inv(-1*Abb)@Abr, np.eye(model.nv-6)))
    J = J_full@Sq_bar
    A = Arr - Arb@inv(Abb)@Abr
    lam = inv(J@inv(A)@J.T)
    Jbar = inv(A) @ J.T @ lam 
    b_sys = getBiasTerms(model, data)
    b = (Jbar).T @ (b_sys[6:] - Arb @ inv(Abb) @ b_sys[:6])
    N = np.eye(model.nv-6) - Jbar @ J
    return lam, J, b, N

def ctrl(model, data):
    kp = 350.
    kd = 50.
    lam, J, b, N = freebaseOSMatrix(model, data)
    (R_des, p_des) = getSitePose(model, data, "target_site")
    (R_arm, p_arm) = getSitePose(model, data, "l_hand_site")
    err_p = p_des - p_arm
    err_rot = 0*rot_err(R_des, R_arm)
    err = np.concatenate([err_p, err_rot])
    # err_dot = np.concatenate([pos_vel, ori_vel])
    xddot_des = kp * err

    tau = J.T @ (lam @ xddot_des + b) - kd * data.qvel[6:]
    return tau
# def getNullSpaceMatrix():
#     return NULL


In [3]:
def osc(model, data):
    # PD control parameters
    kp = 350.
    kd = 50.
    # simulate and render
    ee_pose     = getSitePose(model, data, "ee_site")
    site_pose   = getSitePose(model, data, "target_site")
    jac         = getSiteJacobian(model, data, "ee_site")
    # dq          = calcIKStep(jac, site_pose, ee_pose)
    err_p = site_pose[1] - ee_pose[1]
    err_rot = rot_err(site_pose[0], ee_pose[0])
    # ori_vel, pos_vel = getSiteVel(model, data, "ee_site")

    err = np.concatenate([err_p, err_rot])
    # err_dot = np.concatenate([pos_vel, ori_vel])
    err_dot = jac@data.qvel
    xddot_des = kp * err + kd *(-err_dot)
    mass_matrix = getMassMatrix(model, data)
    bias        = getBiasTerms(model, data)
    op_mass_matrix = getOSMassMatrix(mass_matrix, jac)
    tau = jac.T @ op_mass_matrix@ xddot_des + bias
    return tau

In [4]:
def simulateCtrl(sim, ctrl_method, render=True, t_max=1000):
    if len(sim.model.key_qpos) != 0:
        sim.data.qpos = sim.model.key_qpos
    mujoco.mj_step(sim.model, sim.data)
    log = []
    if render:
        viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
        viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl_method(sim.model, sim.data)
        sim.data.ctrl = u 
        mujoco.mj_step(sim.model, sim.data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()

In [5]:
simulateCtrl(atlas_sim, ctrl)

In [5]:
simulateCtrl(franka_sim, osc)

/home/anon/miniconda3/envs/meng443arm/lib/python3.10/site-packages/glfw/__init__.py:906: GLFWError: (65544) b'Wayland: Standard cursor shape unavailable'
  warnings.warn(message, GLFWError)


Pressed ESC
Quitting.
